download file from here https://www.ipcinfo.org/ipc-country-analysis/population-tracking-tool/en/ file downloaded manually

code adjusted for linux and for higher number of files 

In [3]:
import os
import re
import subprocess
import sys
from datetime import datetime, timedelta
import yt_dlp
import requests
from bs4 import BeautifulSoup
import time
import tempfile
import shutil
import platform

def install_ffmpeg():
    """
    Install ffmpeg based on the operating system.
    Returns the path to ffmpeg if successful, None otherwise.
    """
    system = platform.system().lower()
    
    print("Checking for ffmpeg installation...")
    
    # First check if ffmpeg is already available
    try:
        result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
        if result.returncode == 0:
            print("ffmpeg is already installed and available.")
            return 'ffmpeg'  # Use system ffmpeg
    except FileNotFoundError:
        print("ffmpeg not found. Attempting to install...")
    
    try:
        if system == 'linux':
            # Try different installation methods for Linux
            if os.path.exists('/usr/bin/apt-get'):
                # Debian/Ubuntu
                print("Installing ffmpeg using apt-get...")
                subprocess.run(['sudo', 'apt-get', 'update'], check=False)
                subprocess.run(['sudo', 'apt-get', 'install', '-y', 'ffmpeg'], check=False)
            elif os.path.exists('/usr/bin/yum'):
                # RedHat/CentOS
                print("Installing ffmpeg using yum...")
                subprocess.run(['sudo', 'yum', 'install', '-y', 'ffmpeg'], check=False)
            elif os.path.exists('/usr/bin/pacman'):
                # Arch Linux
                print("Installing ffmpeg using pacman...")
                subprocess.run(['sudo', 'pacman', '-S', '--noconfirm', 'ffmpeg'], check=False)
            else:
                # Try with pip package (static ffmpeg binaries)
                print("Installing ffmpeg-python package...")
                subprocess.run([sys.executable, '-m', 'pip', 'install', 'ffmpeg-python'], check=False)
                
        elif system == 'darwin':
            # macOS
            print("Installing ffmpeg using brew...")
            subprocess.run(['brew', 'install', 'ffmpeg'], check=False)
            
        elif system == 'windows':
            print("For Windows, please download ffmpeg from https://ffmpeg.org/download.html")
            return None
            
    except Exception as e:
        print(f"Error during ffmpeg installation: {e}")
    
    # Check if installation was successful
    try:
        result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
        if result.returncode == 0:
            print("ffmpeg successfully installed!")
            return 'ffmpeg'
    except FileNotFoundError:
        pass
    
    # If system installation failed, try downloading static binaries
    print("Attempting to download static ffmpeg binaries...")
    try:
        import ffmpeg_downloader as ffdl
    except ImportError:
        subprocess.run([sys.executable, '-m', 'pip', 'install', 'ffmpeg-downloader'], check=False)
        try:
            import ffmpeg_downloader as ffdl
            ffmpeg_path = ffdl.ffmpeg_download.get_ffmpeg_exe()
            if ffmpeg_path and os.path.exists(ffmpeg_path):
                print(f"ffmpeg downloaded to: {ffmpeg_path}")
                return ffmpeg_path
        except Exception as e:
            print(f"Failed to download ffmpeg binaries: {e}")
    
    print("Unable to install ffmpeg automatically. Please install it manually.")
    return None

def setup_ffmpeg_for_colab():
    """
    Special setup for Google Colab environment.
    """
    try:
        # Check if we're in Colab
        import google.colab
        print("Detected Google Colab environment. Installing ffmpeg...")
        
        # Install ffmpeg in Colab
        subprocess.run(['apt-get', 'update'], check=False, capture_output=True)
        subprocess.run(['apt-get', 'install', '-y', 'ffmpeg'], check=False, capture_output=True)
        
        # Verify installation
        result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
        if result.returncode == 0:
            print("ffmpeg successfully installed in Colab!")
            return 'ffmpeg'
    except ImportError:
        # Not in Colab
        pass
    except Exception as e:
        print(f"Error setting up ffmpeg in Colab: {e}")
    
    return None

def validate_soundcloud_url(url):
    """Validate if the provided URL is a SoundCloud URL."""
    pattern = r'^https?://(?:www\.)?soundcloud\.com/[\w-]+/[\w-]+'
    return bool(re.match(pattern, url))

def check_if_already_downloaded(url, output_dir):
    """
    Check if a track from the given URL has already been downloaded.
    
    Args:
        url: SoundCloud URL
        output_dir: Directory where files are saved
        
    Returns:
        True if file exists and is valid, False otherwise
    """
    if not os.path.exists(output_dir):
        return False
    
    # Extract expected filename from URL
    # Pattern: idaacadda-DD-MON-YYYY -> IDAACADDA DD-MON-YYYY.mp3
    url_parts = url.split('/')[-1]
    
    # Try to extract date pattern from URL
    date_patterns = [
        r'idaacadda-(\d{1,2})-([a-z]+)-(\d{4})',
        r'(\d{1,2})-([a-z]+)-(\d{4})'
    ]
    
    for pattern in date_patterns:
        match = re.search(pattern, url_parts, re.IGNORECASE)
        if match:
            day, month, year = match.groups()
            # Create possible filenames
            possible_filenames = [
                f"IDAACADDA {int(day):02d}-{month.upper()}-{year}.mp3",
                f"IDAACADDA {int(day)}-{month.upper()}-{year}.mp3",
                f"IDAACADDA {day}-{month.upper()}-{year}.mp3",
                f"IDAACADDA {day.zfill(2)}-{month.upper()}-{year}.mp3"
            ]
            
            for filename in possible_filenames:
                filepath = os.path.join(output_dir, filename)
                if os.path.exists(filepath) and os.path.getsize(filepath) > 1000000:  # > 1MB
                    print(f"File already exists and is valid: {filename}")
                    return True
    
    # Also check for any file that might have been downloaded with yt-dlp's naming
    existing_files = [f for f in os.listdir(output_dir) if f.endswith('.mp3')]
    for file in existing_files:
        # Check if this file might be from this URL
        if url_parts.replace('-', ' ').lower() in file.lower():
            filepath = os.path.join(output_dir, file)
            if os.path.getsize(filepath) > 1000000:  # > 1MB
                print(f"File already exists: {file}")
                return True
    
    return False

def download_soundcloud_audio(url, output_dir="downloads", max_retries=3, ffmpeg_path=None):
    """Download audio from SoundCloud URL with a temp directory approach to prevent partial files."""
    if not validate_soundcloud_url(url):
        print(f"Invalid SoundCloud URL: {url}")
        return False
    
    # Check if already downloaded
    if check_if_already_downloaded(url, output_dir):
        print(f"Skipping download - file already exists for: {url}")
        return True  # Return True since we have the file

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Create a temporary directory for downloads
    with tempfile.TemporaryDirectory() as temp_dir:
        # Configure yt-dlp options
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': os.path.join(temp_dir, '%(title)s.%(ext)s'),
            'noplaylist': True,
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'quiet': False,
            'verbose': False,
            'retries': 10,
            'fragment_retries': 10,
            'skip_unavailable_fragments': False,
            'abort_on_unavailable_fragment': True,
            # Skip metadata download
            'writethumbnail': False,
            'writeinfojson': False,
            'writesubtitles': False,
            'writeautomaticsub': False,
            'writedescription': False,
            'no_write_playlist_metafiles': True,
            'add_metadata': False,
            'embed_thumbnail': False,
            'embed_metadata': False
        }
        
        # Add ffmpeg location if provided
        if ffmpeg_path:
            if ffmpeg_path != 'ffmpeg':  # If it's not the system ffmpeg
                ydl_opts['ffmpeg_location'] = ffmpeg_path
            # Otherwise, let yt-dlp find it in PATH

        # Download with retries
        for attempt in range(max_retries):
            try:
                with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                    info = ydl.extract_info(url, download=True)
                    filename = ydl.prepare_filename(info)
                    base, _ = os.path.splitext(filename)
                    mp3_file = f"{base}.mp3"
                    
                    # Get the path to the MP3 file in temp directory
                    temp_mp3_path = os.path.join(temp_dir, os.path.basename(mp3_file))
                    
                    # Verify the file exists and isn't zero bytes
                    if os.path.exists(temp_mp3_path) and os.path.getsize(temp_mp3_path) > 0:
                        # Move the completed file to the output directory
                        dest_path = os.path.join(output_dir, os.path.basename(temp_mp3_path))
                        shutil.move(temp_mp3_path, dest_path)
                        print(f"Downloaded: {dest_path}")
                        return dest_path
                    else:
                        raise Exception(f"File does not exist or is empty: {temp_mp3_path}")
                        
            except Exception as e:
                print(f"Error downloading {url} (Attempt {attempt+1}/{max_retries}): {str(e)}")
                time.sleep(5 * (attempt + 1))
    
    print(f"Failed to download {url} after {max_retries} attempts")
    return False

def get_soundcloud_urls_by_date_range(profile_url, start_date, end_date):
    """
    Get SoundCloud URLs within a specific date range.
    
    Args:
        profile_url: The SoundCloud profile URL (e.g., "https://soundcloud.com/radio-ergo")
        start_date: Start date (datetime object)
        end_date: End date (datetime object)
        
    Returns:
        A list of SoundCloud URLs within the specified date range
    """
    if not profile_url.endswith('/'):
        profile_url = profile_url + '/'

    try:
        response = requests.get(profile_url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
    except Exception as e:
        print(f"Error fetching profile page: {str(e)}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all track links
    track_urls = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.startswith('/') and profile_url.split('/')[3] in href and '/sets/' not in href:
            full_url = f"https://soundcloud.com{href}"
            if validate_soundcloud_url(full_url):
                track_urls.append(full_url)

    # Filter URLs by date
    urls_in_range = []

    # Month names mapping
    month_names = {
        'january': 1, 'february': 2, 'march': 3, 'april': 4, 'may': 5, 'june': 6,
        'july': 7, 'august': 8, 'september': 9, 'october': 10, 'november': 11, 'december': 12,
        'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'jun': 6, 'jul': 7,
        'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
    }

    for url in track_urls:
        date_match = re.search(r'(\d{1,2})-([a-z]+)-(\d{4})', url.split('/')[-1], re.IGNORECASE)

        if date_match:
            day, month_str, year = date_match.groups()
            month = month_names.get(month_str.lower())

            if month:
                try:
                    track_date = datetime(int(year), month, int(day))
                    if start_date <= track_date <= end_date:
                        urls_in_range.append(url)
                        print(f"Found matching URL: {url}")
                except ValueError:
                    continue
            else:
                print(f"Could not parse month: {month_str} in URL: {url}")
        else:
            print(f"URL does not match date pattern: {url}")

    return urls_in_range

def get_existing_files(output_dir):
    """Get a list of existing complete MP3 files."""
    if not os.path.exists(output_dir):
        return []
    
    mp3_files = [f for f in os.listdir(output_dir) if f.endswith('.mp3') and not f.endswith('.part')]
    
    valid_files = []
    for file in mp3_files:
        file_path = os.path.join(output_dir, file)
        if os.path.getsize(file_path) > 0:
            valid_files.append(file)
    
    return valid_files

def download_by_date_range(profile_url, start_date_str, end_date_str, output_dir=None, save_to_drive=True):
    """
    Download SoundCloud tracks within a date range from earliest to latest date.
    
    Args:
        profile_url: The SoundCloud profile URL
        start_date_str: Start date in format 'YYYY-MM-DD'
        end_date_str: End date in format 'YYYY-MM-DD'
        output_dir: Output directory (default: None, which uses date range as folder name)
        save_to_drive: Whether to save files to Google Drive (default: True)
    """
    # Setup ffmpeg first
    ffmpeg_path = setup_ffmpeg_for_colab()  # Try Colab first
    if not ffmpeg_path:
        ffmpeg_path = install_ffmpeg()  # Try general installation
    
    if not ffmpeg_path:
        print("WARNING: ffmpeg not found. Downloads may fail during audio conversion.")
        print("Please install ffmpeg manually:")
        print("  - Linux/Mac: sudo apt-get install ffmpeg")
        print("  - Windows: Download from https://ffmpeg.org/download.html")
        response = input("Do you want to continue anyway? (y/n): ")
        if response.lower() != 'y':
            return [], []
    
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

    if not output_dir:
        output_dir = f"soundcloud_{start_date_str}_to_{end_date_str}"

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    print(f"Searching for tracks from {start_date_str} to {end_date_str}...")

    existing_files = get_existing_files(output_dir)
    if existing_files:
        print(f"Found {len(existing_files)} existing complete downloads")
        
    # Option 1: Try to find using the profile page
    urls = get_soundcloud_urls_by_date_range(profile_url, start_date, end_date)

    # Option 2: Generate URLs for all days in the range
    manual_urls = []
    current_date = start_date
    while current_date <= end_date:
        day = current_date.day
        month = current_date.strftime('%b').upper()
        year = current_date.year

        # Check if we already have this file
        expected_filename = f"IDAACADDA {day:02d}-{month}-{year}.mp3"
        alt_expected_filename = f"IDAACADDA {day}-{month}-{year}.mp3"
        
        if expected_filename not in existing_files and alt_expected_filename not in existing_files:
            potential_urls = [
                f"https://soundcloud.com/radio-ergo/idaacadda-{day:02d}-{month.lower()}-{year}",
                f"https://soundcloud.com/radio-ergo/idaacadda-{day}-{month.lower()}-{year}"
            ]

            for url in potential_urls:
                # Skip if we already have this file
                if not check_if_already_downloaded(url, output_dir):
                    try:
                        response = requests.head(url)
                        if response.status_code == 200:
                            manual_urls.append((current_date, url))
                            print(f"Manually found URL: {url} for date {current_date.strftime('%Y-%m-%d')}")
                            break
                    except:
                        pass
                else:
                    print(f"Skipping URL (already downloaded): {url}")

        current_date += timedelta(days=1)

    # Create a list of tuples with (date, url) for all URLs
    dated_urls = []
    
    # Process URLs from profile page - extract dates from them
    for url in urls:
        # Skip if already downloaded
        if check_if_already_downloaded(url, output_dir):
            print(f"Skipping URL (already downloaded): {url}")
            continue
            
        date_match = re.search(r'(\d{1,2})-([a-z]+)-(\d{4})', url.split('/')[-1], re.IGNORECASE)
        if date_match:
            day, month_str, year = date_match.groups()
            month_names = {
                'january': 1, 'february': 2, 'march': 3, 'april': 4, 'may': 5, 'june': 6,
                'july': 7, 'august': 8, 'september': 9, 'october': 10, 'november': 11, 'december': 12,
                'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'jun': 6, 'jul': 7,
                'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
            }
            month = month_names.get(month_str.lower())
            if month:
                try:
                    track_date = datetime(int(year), month, int(day))
                    dated_urls.append((track_date, url))
                except ValueError:
                    dated_urls.append((end_date, url))
            else:
                dated_urls.append((end_date, url))
        else:
            dated_urls.append((end_date, url))
    
    # Combine both lists of dated URLs
    all_dated_urls = dated_urls + manual_urls
    
    # Remove duplicates while preserving order
    seen_urls = set()
    unique_dated_urls = []
    for date, url in all_dated_urls:
        if url not in seen_urls:
            seen_urls.add(url)
            unique_dated_urls.append((date, url))
    
    # Sort by date (earliest first)
    unique_dated_urls.sort(key=lambda x: x[0])
    
    # Extract just the URLs in order
    all_urls = [url for _, url in unique_dated_urls]

    if not all_urls:
        print("No new tracks found to download.")
        return [], []

    print(f"Found {len(all_urls)} tracks to download.")
    print("URLs to process (from earliest to latest):")
    for date, url in unique_dated_urls:
        print(f"- {date.strftime('%Y-%m-%d')}: {url}")

    print(f"\nStarting download of {len(all_urls)} SoundCloud audios to: {output_dir}")
    downloaded_files = []
    failed_urls = []
    
    for url in all_urls:
        try:
            response = requests.head(url)
            if response.status_code == 404:
                print(f"Skipping URL with 404 status: {url}")
                continue
        except Exception as e:
            print(f"Error checking URL {url}: {str(e)}")
            continue
            
        result = download_soundcloud_audio(url, output_dir, ffmpeg_path=ffmpeg_path)
        if result:
            downloaded_files.append(result)
        else:
            failed_urls.append(url)
            
        time.sleep(2)

    print("\nSummary:")
    print(f"Total URLs processed: {len(all_urls)}")
    print(f"Successfully downloaded: {len(downloaded_files)}")
    print(f"Failed downloads: {len(failed_urls)}")
    
    if downloaded_files:
        print("\nDownloaded files:")
        for file in downloaded_files:
            print(f"- {file}")
            
    if failed_urls:
        print("\nFailed URLs:")
        for url in failed_urls:
            print(f"- {url}")

    if save_to_drive and downloaded_files:
        drive_path = "/content/drive/MyDrive/" + output_dir
        print(f"\nSaving files to Google Drive at: {drive_path}")

        if not os.path.exists(drive_path):
            os.makedirs(drive_path)

        os.system(f"cp -r {output_dir}/*.mp3 {drive_path}/")
        print(f"Files successfully saved to Google Drive")

    return downloaded_files, failed_urls

def retry_failed_downloads(failed_urls, output_dir, save_to_drive=False, max_retries=5):
    """
    Retry downloading failed URLs with protection against infinite loops.
    
    Args:
        failed_urls: List of URLs that failed to download in the first attempt
        output_dir: Directory to save downloaded files
        save_to_drive: Whether to save files to Google Drive
        max_retries: Maximum number of retry attempts per URL
        
    Returns:
        Tuple of (downloaded_files, still_failed_urls)
    """
    if not failed_urls:
        print("No failed downloads to retry.")
        return [], []
    
    # Setup ffmpeg if not already done
    ffmpeg_path = setup_ffmpeg_for_colab() or install_ffmpeg()
        
    print(f"Retrying {len(failed_urls)} failed downloads...")
    
    downloaded_files = []
    still_failed = []
    
    # Track retry attempts for each URL to prevent infinite loops
    retry_counts = {}
    max_retry_attempts = 1  # Only try once more to avoid excessive retries
    
    for url in failed_urls:
        # First check if file was actually downloaded in previous attempts
        if check_if_already_downloaded(url, output_dir):
            print(f"File already exists for: {url}")
            downloaded_files.append(url)
            continue
            
        # Check if we've already tried this URL too many times
        retry_counts[url] = retry_counts.get(url, 0) + 1
        
        if retry_counts[url] > max_retry_attempts:
            print(f"Skipping {url} - exceeded maximum retry attempts")
            still_failed.append(url)
            continue
            
        print(f"Retrying: {url} (Attempt {retry_counts[url]}/{max_retry_attempts})")
        result = download_soundcloud_audio(url, output_dir, max_retries=max_retries, ffmpeg_path=ffmpeg_path)
        
        if result:
            downloaded_files.append(result)
            print(f"Successfully downloaded on retry: {url}")
        else:
            still_failed.append(url)
            print(f"Failed again: {url}")
            
        time.sleep(5)
        
    print("\nRetry Summary:")
    print(f"Successfully downloaded: {len(downloaded_files)}")
    print(f"Still failed: {len(still_failed)}")
    
    if downloaded_files and save_to_drive:
        drive_path = "/content/drive/MyDrive/" + output_dir
        print(f"\nSaving files to Google Drive at: {drive_path}")
        
        if not os.path.exists(drive_path):
            os.makedirs(drive_path)
            
        os.system(f"cp -r {output_dir}/*.mp3 {drive_path}/")
        print("Files successfully saved to Google Drive")
        
    return downloaded_files, still_failed



In [4]:
# Example usage
if __name__ == "__main__":
    profile_url = "https://soundcloud.com/radio-ergo"
    start_date = "2024-07-01"
    end_date = "2024-07-02"
    output_dir = "data/soundcloud_2024-07-01_to_2024-09-30"
    save_to_drive = False
    max_retries = 1

    # Download files
    downloaded_files, failed_urls = download_by_date_range(
        profile_url, 
        start_date, 
        end_date, 
        output_dir=output_dir,
        save_to_drive=save_to_drive
    )

    # Retry failed downloads if any
    if failed_urls:
        print("\nRetrying failed downloads...")
        retry_downloaded, still_failed = retry_failed_downloads(
            failed_urls, 
            output_dir, 
            save_to_drive=save_to_drive,
            max_retries=max_retries
        )

Checking for ffmpeg installation...
ffmpeg not found. Attempting to install...
Installing ffmpeg using apt-get...


Hit:1 https://packages.cloud.google.com/apt cloud-sdk InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://download.docker.com/linux/ubuntu focal InRelease
Hit:4 https://us-east-1.ec2.archive.ubuntu.com/ubuntu focal InRelease
Hit:5 https://us-east-1.ec2.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 https://us-east-1.ec2.archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:7 http://deb.wakemeops.com/wakemeops stable InRelease
Hit:8 https://archive.ubuntu.com/ubuntu focal InRelease
Hit:9 https://security.ubuntu.com/ubuntu focal-security InRelease
Hit:10 https://cloud.archive.ubuntu.com/ubuntu focal InRelease
Hit:11 https://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:12 https://cloud.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:13 https://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:14 https://cloud.archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:15 https://cloud.archive.ubuntu.com/ubuntu focal-security InRelease

debconf: delaying package configuration, since apt-utils is not installed


Fetched 21.3 MB in 3s (7950 kB/s)
Selecting previously unselected package libslang2:amd64.
(Reading database ... 121236 files and directories currently installed.)
Preparing to unpack .../00-libslang2_2.3.2-4_amd64.deb ...
Unpacking libslang2:amd64 (2.3.2-4) ...
Selecting previously unselected package libsodium23:amd64.
Preparing to unpack .../01-libsodium23_1.0.18-1_amd64.deb ...
Unpacking libsodium23:amd64 (1.0.18-1) ...
Selecting previously unselected package libusb-1.0-0:amd64.
Preparing to unpack .../02-libusb-1.0-0_2%3a1.0.23-2build1_amd64.deb ...
Unpacking libusb-1.0-0:amd64 (2:1.0.23-2build1) ...
Selecting previously unselected package libraw1394-11:amd64.
Preparing to unpack .../03-libraw1394-11_2.1.2-1_amd64.deb ...
Unpacking libraw1394-11:amd64 (2.1.2-1) ...
Selecting previously unselected package libavc1394-0:amd64.
Preparing to unpack .../04-libavc1394-0_0.5.4-5_amd64.deb ...
Unpacking libavc1394-0:amd64 (0.5.4-5) ...
Selecting previously unselected package libass9:amd64.


[ExtractAudio] Destination: /tmp/tmpyf6fu8o0/IDAACADDA 01-JUL-2024.mp3


...

ERROR: Postprocessing: audio conversion failed: ffmpeg not found. Please install or provide the path using --ffmpeg-location


Error downloading https://soundcloud.com/radio-ergo/idaacadda-01-jul-2024 (Attempt 1/1): ERROR: Postprocessing: audio conversion failed: ffmpeg not found. Please install or provide the path using --ffmpeg-location
Failed to download https://soundcloud.com/radio-ergo/idaacadda-01-jul-2024 after 1 attempts
Failed again: https://soundcloud.com/radio-ergo/idaacadda-01-jul-2024
Retrying: https://soundcloud.com/radio-ergo/idaacadda-02-jul-2024 (Attempt 1/1)
[soundcloud] Extracting URL: https://soundcloud.com/radio-ergo/idaacadda-02-jul-2024
[soundcloud] radio-ergo/idaacadda-02-jul-2024: Downloading info JSON
[soundcloud] 1860877620: Downloading hls_mp3 format info JSON
[soundcloud] 1860877620: Downloading http_mp3 format info JSON
[soundcloud] 1860877620: Downloading hls_opus format info JSON
[info] 1860877620: Downloading 1 format(s): hls_opus_0_0
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 359
[download] Destination: /tmp/tmplaqlfb58/IDAACADDA 02-JUL-2024.opus
[downl

[ExtractAudio] Destination: /tmp/tmplaqlfb58/IDAACADDA 02-JUL-2024.mp3


ERROR: Postprocessing: audio conversion failed: ffmpeg not found. Please install or provide the path using --ffmpeg-location


Error downloading https://soundcloud.com/radio-ergo/idaacadda-02-jul-2024 (Attempt 1/1): ERROR: Postprocessing: audio conversion failed: ffmpeg not found. Please install or provide the path using --ffmpeg-location
Failed to download https://soundcloud.com/radio-ergo/idaacadda-02-jul-2024 after 1 attempts
Failed again: https://soundcloud.com/radio-ergo/idaacadda-02-jul-2024

Retry Summary:
Successfully downloaded: 0
Still failed: 2


In [10]:
#!/usr/bin/env python3
"""
SoundCloud Downloader for Linux
Handles ffmpeg installation and prevents re-downloading existing files
"""

import os
import re
import subprocess
import sys
from datetime import datetime, timedelta
import yt_dlp
import requests
from bs4 import BeautifulSoup
import time
import hashlib
import json

class SoundCloudDownloader:
    def __init__(self, output_dir="downloads"):
        self.output_dir = output_dir
        self.ffmpeg_path = None
        self.download_log = {}
        self.log_file = os.path.join(output_dir, ".download_log.json")
        
        # Create output directory if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        # Load download log
        self.load_download_log()
        
        # Setup ffmpeg on initialization
        self.setup_ffmpeg()
    
    def load_download_log(self):
        """Load the download log from file."""
        if os.path.exists(self.log_file):
            try:
                with open(self.log_file, 'r') as f:
                    self.download_log = json.load(f)
            except:
                self.download_log = {}
    
    def save_download_log(self):
        """Save the download log to file."""
        try:
            with open(self.log_file, 'w') as f:
                json.dump(self.download_log, f, indent=2)
        except Exception as e:
            print(f"Warning: Could not save download log: {e}")
    
    def setup_ffmpeg(self):
        """Setup ffmpeg for Linux systems."""
        print("Checking for ffmpeg installation...")
        
        # First check if ffmpeg is already in PATH
        ffmpeg_locations = [
            'ffmpeg',
            '/usr/bin/ffmpeg',
            '/usr/local/bin/ffmpeg',
            '/opt/homebrew/bin/ffmpeg'
        ]
        
        for location in ffmpeg_locations:
            if self.check_ffmpeg(location):
                self.ffmpeg_path = location
                print(f"✓ Found ffmpeg at: {location}")
                return True
        
        # If not found, try to install it
        print("ffmpeg not found. Attempting to install...")
        
        # Detect the Linux distribution
        distro = self.detect_linux_distro()
        
        if distro in ['ubuntu', 'debian']:
            if self.install_ffmpeg_debian():
                return True
        elif distro in ['fedora', 'rhel', 'centos']:
            if self.install_ffmpeg_fedora():
                return True
        elif distro == 'arch':
            if self.install_ffmpeg_arch():
                return True
        
        # Try generic installation as last resort
        if self.install_ffmpeg_generic():
            return True
        
        print("⚠ Failed to install ffmpeg automatically.")
        print("Please install it manually:")
        print("  Ubuntu/Debian: sudo apt-get install ffmpeg")
        print("  Fedora: sudo dnf install ffmpeg")
        print("  Arch: sudo pacman -S ffmpeg")
        return False
    
    def detect_linux_distro(self):
        """Detect the Linux distribution."""
        try:
            with open('/etc/os-release', 'r') as f:
                content = f.read().lower()
                if 'ubuntu' in content or 'debian' in content:
                    return 'ubuntu'
                elif 'fedora' in content:
                    return 'fedora'
                elif 'arch' in content:
                    return 'arch'
                elif 'centos' in content or 'rhel' in content:
                    return 'centos'
        except:
            pass
        return 'unknown'
    
    def check_ffmpeg(self, path):
        """Check if ffmpeg exists and works at the given path."""
        try:
            result = subprocess.run(
                [path, '-version'],
                capture_output=True,
                text=True,
                timeout=5
            )
            return result.returncode == 0
        except:
            return False
    
    def install_ffmpeg_debian(self):
        """Install ffmpeg on Debian/Ubuntu systems."""
        try:
            # Update package lists
            subprocess.run(['sudo', 'apt-get', 'update'], check=False, capture_output=True)
            
            # Install ffmpeg
            result = subprocess.run(
                ['sudo', 'apt-get', 'install', '-y', 'ffmpeg'],
                capture_output=True,
                text=True
            )
            
            if result.returncode == 0:
                self.ffmpeg_path = '/usr/bin/ffmpeg'
                print("✓ Successfully installed ffmpeg via apt-get")
                return True
        except Exception as e:
            print(f"Failed to install via apt-get: {e}")
        
        return False
    
    def install_ffmpeg_fedora(self):
        """Install ffmpeg on Fedora/RHEL systems."""
        try:
            # Enable RPM Fusion repository first
            subprocess.run([
                'sudo', 'dnf', 'install', '-y',
                'https://download1.rpmfusion.org/free/fedora/rpmfusion-free-release-$(rpm -E %fedora).noarch.rpm'
            ], check=False, capture_output=True)
            
            # Install ffmpeg
            result = subprocess.run(
                ['sudo', 'dnf', 'install', '-y', 'ffmpeg'],
                capture_output=True,
                text=True
            )
            
            if result.returncode == 0:
                self.ffmpeg_path = '/usr/bin/ffmpeg'
                print("✓ Successfully installed ffmpeg via dnf")
                return True
        except Exception as e:
            print(f"Failed to install via dnf: {e}")
        
        return False
    
    def install_ffmpeg_arch(self):
        """Install ffmpeg on Arch Linux systems."""
        try:
            result = subprocess.run(
                ['sudo', 'pacman', '-S', '--noconfirm', 'ffmpeg'],
                capture_output=True,
                text=True
            )
            
            if result.returncode == 0:
                self.ffmpeg_path = '/usr/bin/ffmpeg'
                print("✓ Successfully installed ffmpeg via pacman")
                return True
        except Exception as e:
            print(f"Failed to install via pacman: {e}")
        
        return False
    
    def install_ffmpeg_generic(self):
        """Try to install ffmpeg using snap as a fallback."""
        try:
            # Check if snap is available
            snap_check = subprocess.run(['which', 'snap'], capture_output=True)
            if snap_check.returncode == 0:
                result = subprocess.run(
                    ['sudo', 'snap', 'install', 'ffmpeg'],
                    capture_output=True,
                    text=True
                )
                
                if result.returncode == 0:
                    self.ffmpeg_path = '/snap/bin/ffmpeg'
                    print("✓ Successfully installed ffmpeg via snap")
                    return True
        except:
            pass
        
        return False
    
    def validate_url(self, url):
        """Validate if the URL is a valid SoundCloud URL."""
        pattern = r'^https?://(?:www\.)?soundcloud\.com/[\w-]+/[\w-]+'
        return bool(re.match(pattern, url))
    
    def get_file_hash(self, url):
        """Generate a unique hash for a URL."""
        return hashlib.md5(url.encode()).hexdigest()
    
    def is_already_downloaded(self, url):
        """Check if a URL has already been successfully downloaded."""
        url_hash = self.get_file_hash(url)
        
        if url_hash in self.download_log:
            file_path = self.download_log[url_hash].get('file_path')
            if file_path and os.path.exists(file_path):
                file_size = os.path.getsize(file_path)
                if file_size > 1000000:  # > 1MB
                    print(f"✓ Already downloaded: {os.path.basename(file_path)}")
                    return True
        
        return False
    
    def download_audio(self, url, max_retries=3):
        """Download audio from SoundCloud URL."""
        if not self.validate_url(url):
            print(f"✗ Invalid URL: {url}")
            return None
        
        # Check if already downloaded
        if self.is_already_downloaded(url):
            return self.download_log[self.get_file_hash(url)]['file_path']
        
        print(f"⬇ Downloading: {url}")
        
        # Configure yt-dlp options
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': os.path.join(self.output_dir, '%(title)s.%(ext)s'),
            'noplaylist': True,
            'quiet': True,
            'no_warnings': True,
            'extract_flat': False,
            'retries': 10,
            'fragment_retries': 10,
            'ignoreerrors': False,
            'abort_on_unavailable_fragment': False,
            'skip_unavailable_fragments': True,
            # Audio extraction settings
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            # Add metadata
            'add_metadata': True,
            'embed_metadata': True,
            'embed_thumbnail': False,
            'writethumbnail': False,
        }
        
        # Add ffmpeg location if we have it
        if self.ffmpeg_path and self.ffmpeg_path != 'ffmpeg':
            ydl_opts['ffmpeg_location'] = self.ffmpeg_path
        
        # Try downloading with retries
        for attempt in range(max_retries):
            try:
                with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                    # Extract info first
                    info = ydl.extract_info(url, download=False)
                    
                    # Generate expected filename
                    filename = ydl.prepare_filename(info)
                    base, _ = os.path.splitext(filename)
                    mp3_file = f"{base}.mp3"
                    
                    # Check if file already exists
                    if os.path.exists(mp3_file) and os.path.getsize(mp3_file) > 1000000:
                        print(f"✓ File already exists: {os.path.basename(mp3_file)}")
                        # Add to log
                        self.download_log[self.get_file_hash(url)] = {
                            'url': url,
                            'file_path': mp3_file,
                            'download_date': datetime.now().isoformat()
                        }
                        self.save_download_log()
                        return mp3_file
                    
                    # Now download
                    ydl.download([url])
                    
                    # Verify the download
                    if os.path.exists(mp3_file) and os.path.getsize(mp3_file) > 0:
                        print(f"✓ Successfully downloaded: {os.path.basename(mp3_file)}")
                        
                        # Add to download log
                        self.download_log[self.get_file_hash(url)] = {
                            'url': url,
                            'file_path': mp3_file,
                            'download_date': datetime.now().isoformat()
                        }
                        self.save_download_log()
                        
                        return mp3_file
                    else:
                        raise Exception("Downloaded file is empty or missing")
                        
            except Exception as e:
                print(f"  Attempt {attempt + 1}/{max_retries} failed: {str(e)[:100]}")
                if attempt < max_retries - 1:
                    time.sleep(5 * (attempt + 1))
        
        print(f"✗ Failed to download after {max_retries} attempts")
        return None
    
    def download_date_range(self, profile_url, start_date, end_date):
        """
        Download tracks from a SoundCloud profile within a date range.
        
        Args:
            profile_url: SoundCloud profile URL
            start_date: Start date (datetime object or string 'YYYY-MM-DD')
            end_date: End date (datetime object or string 'YYYY-MM-DD')
        
        Returns:
            Tuple of (successful_downloads, failed_urls)
        """
        # Parse dates if strings
        if isinstance(start_date, str):
            start_date = datetime.strptime(start_date, '%Y-%m-%d')
        if isinstance(end_date, str):
            end_date = datetime.strptime(end_date, '%Y-%m-%d')
        
        print(f"\n📅 Searching for tracks from {start_date.date()} to {end_date.date()}")
        print(f"📁 Output directory: {self.output_dir}\n")
        
        # Generate URLs for date range
        urls = self.generate_urls_for_range(profile_url, start_date, end_date)
        
        if not urls:
            print("No URLs found for the specified date range")
            return [], []
        
        print(f"Found {len(urls)} potential tracks to download\n")
        
        successful = []
        failed = []
        
        for i, (date, url) in enumerate(urls, 1):
            print(f"[{i}/{len(urls)}] Processing {date.date()}")
            
            # Check if URL exists
            try:
                response = requests.head(url, timeout=5)
                if response.status_code == 404:
                    print(f"  ✗ URL not found (404)")
                    continue
            except:
                pass  # Try downloading anyway
            
            result = self.download_audio(url)
            
            if result:
                successful.append(result)
            else:
                failed.append(url)
            
            # Small delay between downloads
            if i < len(urls):
                time.sleep(2)
        
        # Print summary
        print(f"\n{'='*50}")
        print(f"Download Summary:")
        print(f"  ✓ Successful: {len(successful)}")
        print(f"  ✗ Failed: {len(failed)}")
        print(f"  📁 Files saved to: {self.output_dir}")
        print(f"{'='*50}\n")
        
        return successful, failed
    
    def generate_urls_for_range(self, profile_url, start_date, end_date):
        """Generate potential URLs for a date range."""
        urls = []
        
        # Clean profile URL
        profile_url = profile_url.rstrip('/')
        username = profile_url.split('/')[-1]
        
        current_date = start_date
        while current_date <= end_date:
            day = current_date.day
            month = current_date.strftime('%b').lower()
            year = current_date.year
            
            # Generate potential URL formats
            potential_urls = [
                f"{profile_url}/idaacadda-{day:02d}-{month}-{year}",
                f"{profile_url}/idaacadda-{day}-{month}-{year}",
                f"{profile_url}/idaacada-{day:02d}-{month}-{year}",  # Typo variant
                f"{profile_url}/idaacada-{day}-{month}-{year}",  # Typo variant
            ]
            
            # Add the first valid URL for this date
            for url in potential_urls:
                urls.append((current_date, url))
                break  # Only add one URL per date
            
            current_date += timedelta(days=1)
        
        return urls


def main():
    """Main function for command-line usage."""
    # Example usage
    downloader = SoundCloudDownloader(output_dir="soundcloud_downloads")
    
    # Download a specific date range
    profile_url = "https://soundcloud.com/radio-ergo"
    start_date = "2024-07-01"
    end_date = "2024-07-02"
    
    successful, failed = downloader.download_date_range(
        profile_url,
        start_date,
        end_date
    )
    
    # Retry failed downloads if any
    if failed:
        print("\n🔄 Retrying failed downloads...")
        retry_successful = []
        still_failed = []
        
        for url in failed:
            result = downloader.download_audio(url, max_retries=0)
            if result:
                retry_successful.append(result)
            else:
                still_failed.append(url)
        
        if retry_successful:
            print(f"✓ Successfully downloaded {len(retry_successful)} on retry")
        if still_failed:
            print(f"✗ Still failed: {len(still_failed)} URLs")
            for url in still_failed:
                print(f"  - {url}")


if __name__ == "__main__":
    main()

Checking for ffmpeg installation...
✓ Found ffmpeg at: ffmpeg

📅 Searching for tracks from 2024-07-01 to 2024-07-02
📁 Output directory: soundcloud_downloads

Found 2 potential tracks to download

[1/2] Processing 2024-07-01


⬇ Downloading: https://soundcloud.com/radio-ergo/idaacadda-01-jul-2024


ERROR: Postprocessing: audio conversion failed: ffmpeg not found. Please install or provide the path using --ffmpeg-location


  Attempt 1/3 failed: ERROR: Postprocessing: audio conversion failed: ffmpeg not found. Please install or provide the path


ERROR: Postprocessing: audio conversion failed: ffmpeg not found. Please install or provide the path using --ffmpeg-location


...

  Attempt 1/3 failed: ERROR: Postprocessing: audio conversion failed: ffmpeg not found. Please install or provide the path


ERROR: Postprocessing: audio conversion failed: ffmpeg not found. Please install or provide the path using --ffmpeg-location


  Attempt 2/3 failed: ERROR: Postprocessing: audio conversion failed: ffmpeg not found. Please install or provide the path


ERROR: Postprocessing: audio conversion failed: ffmpeg not found. Please install or provide the path using --ffmpeg-location


  Attempt 3/3 failed: ERROR: Postprocessing: audio conversion failed: ffmpeg not found. Please install or provide the path
✗ Failed to download after 3 attempts

Download Summary:
  ✓ Successful: 0
  ✗ Failed: 2
  📁 Files saved to: soundcloud_downloads


🔄 Retrying failed downloads...
⬇ Downloading: https://soundcloud.com/radio-ergo/idaacadda-01-jul-2024
✗ Failed to download after 0 attempts
⬇ Downloading: https://soundcloud.com/radio-ergo/idaacadda-02-jul-2024
✗ Failed to download after 0 attempts
✗ Still failed: 2 URLs
  - https://soundcloud.com/radio-ergo/idaacadda-01-jul-2024
  - https://soundcloud.com/radio-ergo/idaacadda-02-jul-2024
